In [67]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas_datareader import data

# ^BVSP - Índice Bovespa
# ISEE - Índice de Sustentabilidade Empresarial
# BNO - United States Brent Oil Fund
# PETR3 - Petrobrás
start_date = '2021-03-19'
end_date = '2021-04-01'
tickers = ['^BVSP','ISEE','BNO','PETR3.SA']
panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)

In [68]:
mystocks = pd.DataFrame()

mystocks = panel_data[[('Close', '^BVSP'),('Close', 'ISEE'),('Close', 'BNO'),('Close', 'PETR3.SA')]]
mystocks.columns = ['IBOV','ISEE','BNO','PETR3']

mystocks = mystocks.dropna()
mystocks.reset_index(inplace=True)

In [69]:
mystocks.head()

,Date,IBOV,ISEE,BNO,PETR3
0,2021-03-19,116222.0,6.45,16.260000,23.520000
1,2021-03-22,114979.0,6.41,16.230000,23.080000
2,2021-03-23,113262.0,6.24,15.310000,22.549999
3,2021-03-24,112064.0,5.85,16.139999,22.530001
4,2021-03-25,113750.0,6.30,15.570000,22.959999


In [129]:
ts_IBOV = mystocks.IBOV / mystocks.IBOV.max()
ts_ISEE = mystocks.ISEE / mystocks.ISEE.max()
ts_BNO = mystocks.BNO /mystocks.BNO.max()
ts_PETR3 = mystocks.PETR3 / mystocks.PETR3.max()

In [130]:
ts_PETR3

0    0.982456
1    0.964077
2    0.941938
3    0.941103
4    0.959064
5    0.973266
6    0.987886
7    0.984962
8    1.000000
9    0.986633
Name: PETR3, dtype: float64

In [131]:
import torch
import torch.nn as nn
from torch.autograd import Variable

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [132]:
dados = torch.tensor(ts_PETR3)
dados = dados.to(device)
dados = dados.type(torch.float) 

EXAMPLE 1 - Regression Network Archtecure

In [133]:
input_size = 1 # Número de variáveis de dados sequênciais
n_hidden = 100 # Número de 'hiden nodes' na camada LSTM (Long-Short Term Memory)
n_layers = 2   # Número de camadas do modelo LSTM
out_size = 1   # O tamanho da saída desejada do RNN (Recurrent Neural Network)

lstm = nn.LSTM(input_size, n_hidden, n_layers)
linear_fullConnected = nn.Linear(n_hidden, 1)

In [134]:
dados

tensor([0.9825, 0.9641, 0.9419, 0.9411, 0.9591, 0.9733, 0.9879, 0.9850, 1.0000,
        0.9866])

In [135]:
# def get_batches(data, window):
def get_batches(data, lenght_windows):
  lenght_data = data.shape[0]
  data_x = torch.zeros(lenght_data - lenght_windows-1, lenght_windows)
  data_y =  torch.zeros(lenght_data - lenght_windows-1)
  print(data_x.shape)
  print(data_y.shape)
  for i in range(lenght_windows):
    data_x[:,i] = data[i:lenght_data - lenght_windows -1 + i]
    print('loop', i)
#     print(data_x[:,i].shape)
    # print(data_x[:,i])
  
  # Uma dimensão adicional é requerida
  print("data_Y",data_y.shape)
  data_y = data[lenght_windows:].unsqueeze(-1)
  print("data_Y after unsqueeze",data_y.shape)
  # print(data_y)
  return data_x.unsqueeze(-1), data_y 

In [136]:
# Data Flow Protocol:
# 1. Network input shape: (batch_size, seq_length, num_features)
# 2. LSTM output shape: (batch_size, seq_length, hidden_size)
# 3. Linear input shape: (batch_size * seq_length, hidden_size)
# 4. Linear output: (batch size * seq_length, out_size)

train_size = int(len(dados) * 0.8)
test_size = len(dados) - train_size

seq_length = 6
x, y = get_batches(dados,3)

x, y = x.to(device), y.to(device) # importante para o uso de GPU

trainX, trainY  = x[:train_size],y[:train_size]
testX, testY = x[train_size:],y[train_size:]
# lstm_out, hs = lstm(x)

torch.Size([6, 3])
torch.Size([6])
loop 0
loop 1
loop 2
data_Y torch.Size([6])
data_Y after unsqueeze torch.Size([7, 1])


In [137]:
out, (h_out, cn) = lstm(trainX)

In [138]:
last_layer = h_out[-1]
out_final = linear_fullConnected(last_layer.view(-1, n_hidden))

In [139]:
out_final.shape

torch.Size([3, 1])

In [140]:
out_final

tensor([[0.0281],
        [0.0281],
        [0.0281]], grad_fn=<AddmmBackward>)